In [1]:
import re, json
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from tqdm import tqdm

from os import listdir
from os.path import isfile, join

In [2]:
mypath = '../forums/medlinks-forum/medlinks/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
delete_sections = ['ATTENTION', 'ATTACHMENTS', 'LISTOFREFERENCES', 'CLASSIFICATION']
delete_sections = set(delete_sections)

words = ['мм', 'рт', 'ст', 'г', 'м', 'мл', 'кг', 'мг', 'см', 'ч', 'е', 'х', 'a', 'd', 'b', 'мц', 'лг', 'бт',
        'млн', 'фсг'] 
set_stopWords = stopwords.words("russian") + words
set_stopWords = set(set_stopWords)

In [4]:
def deleteStopWords(text):
    stop = ['др. греч.', 'греч.', 'лат.', 'др.', 'и т. д', 'и т.д', 'т. е.', 'т.е.', 'т.н.', 'т. н.', 'т.п.', 'т. п.',
                'т. д', 'т.д', 'мм. рт. ст.', 'мм рт. ст.', 'мм.рт.ст', 'МЕДПОИСК.РУ', 'Контактная форма ниже', 'ºС']
    for i in stop:
        text = text.replace(i, '')
    return text

In [5]:
def clearText(text):
    text = deleteStopWords(text)
    text = text.replace('\xad', '')
    text = text.replace('\n\n', '.')
    text = text.replace('..', '.')
    text = text.replace(':', ' ')
    text = text.replace(';', '.')
    text = text.replace('?', '.')
    text = text.replace('!', '.')
    text = re.sub(r'\[[\d+]*\]', '', text)
    text = re.sub(r'\([\d+]*\)', '', text)
    text = re.sub('\d+', ' ', text)
    text = re.sub('[^\w.]', ' ', text)
    
    final_s = ''
    for sen in text.split('.'):
        sen = sen.strip()
        tmp_t = ''
        for tok in sen.split(' '):
            tok = tok.strip()
            if(len(tok) <= 2):
                continue
            if(tok.lower() not in set_stopWords):
                tmp_t += tok + ' '
        if(len(tmp_t.split()) < 2):
            continue
        final_s += tmp_t + '.'
    return final_s

In [6]:
def getFile(fileName):
    data = json.load(open(mypath+fileName))
    topics = data.keys()
    final_s = ''
    for topic in topics:
        for i in data[topic]:
            tmp = clearText(i)
            final_s += tmp
    return final_s

In [7]:
allText = ''
for file in tqdm(onlyfiles):
    fileText = getFile(file)
    allText += fileText

100%|██████████| 58/58 [00:44<00:00,  1.30it/s]


In [8]:
len(allText)

106633240

In [9]:
# with open('medlinks-forum.txt', 'w+') as f:
#     f.write(allText)